In [2]:
# Import libraries
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, roc_curve, roc_auc_score, confusion_matrix

In [3]:
# Load dataset
df = pd.read_csv('Dataset/cleaned_adult.csv')

In [4]:
features = df.drop(columns=['income']) 
label = df['income'] #predict 

In [6]:
# Apply Standard Scaler for the features
scaler = StandardScaler()
features = pd.DataFrame(scaler.fit_transform(features), columns = features.columns)

In [7]:
# Split the dataset into training(80%) and testing(20%) sets
X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.2, random_state=42)